<div style="background-color: #042354;padding-top: 10px;padding-bottom: 10px;padding-left: 10px;">
    <img src="https://assets.website-files.com/60b56cdf18d38e15ce088579/60b580ab8d45223dc593968d_pucp%7D.svg" style="float: left"/><br /><br />
    <h1 style="color:white">
        1MTR15 Temas de Ingeniería Mecatrónica D
    </h1>
</div>

<div style="background-color: #DDDDDD;padding-top: 10px;padding-bottom: 10px;padding-left: 10px;">
    <h2>Lab 7: [AWS] API Gateway, Lambda, DynamoDB, IAM, S3</h2>
</div>

__Instructor:__ Jorge Bustamante

### I. Link the API POST method and its Lambda function to the DynamoDB table.

<div class="alert alert-block alert-info">
    As a reminder, from now on <b>[CODE]</b> stands for your student code (e.g., 20201234).
</div>

1. In __Lambda__, modify the  __`cy-store-data-[CODE]`__ function as bellow:

In [ ]:
const AWS = require('aws-sdk');
const dynamodb = new AWS.DynamoDB({region:'sa-east-1',apiVersion:'2012-08-10'});

exports.handler = (event, context, callback) => {
    const params = {
        Item: {
            "Userid": {
                "S": "U" + Math.random()
            },
            "Age": {
                "N": "" + event.age
            },
            "Height": {
                "N": "" + event.height
            },
            "Income": { 
                "N": "" + event.income
            }
        },
        TableName: "compare-yourself-1mtr15"
    }

    dynamoDB.putItem(params, function(err, data) {
        if (!err) {
            console.log(data);
            callback(null, data);
        } else {
            console.log(err);
            callback(err);
        }
    })
}

2. __Deploy__ the function.<br /><br />
3. Perform persistence tests from __Lambda__, __API Gateway__ and __Postman__.<br /><br />
    3.1 Test from __Lambda__.
    - Create a __test event__ with the following JSON format:

In [ ]:
{
    "age": "20",
    "height": "170",
    "income": "10000"
}

- Switch to __DynamoDB__ and check the items of the table __`compare-yourself-[CODE]`__ previously created. The values sent from the __`cy-store-data-[CODE]`__ function must be reflected.

    3.2. Test from __API Gateway__.<br /><br />
    - Go to __POST__ method > __Integration Request__ > Mapping templates > Content-Type: application-json, and check for this template:

In [ ]:
{
    "age": $input.json('$.age'),
    "height": $input.json('$.height'),
    "income": $input.json('$.income')
}

- Go to `POST method > Integration Response > Mapping templates > Content-Type: application-json`, and check for an empty template.<br /><br/>
- Click __Test__ in __Method Execution__ with the following __Request Body__:

In [ ]:
{
    "age": "30",
    "height": "180",
    "income": "20000"
}

- Switch to __DynamoDB__ and check the items of the table __`compare-yourself-[CODE]`__ previously created. The values sent from the __POST__ method in __API Gateway__ must be reflected.

    3.3. Test from __Postman__.<br/><br/>
    - Deploy API.<br/><br/>
    - Open Postman, create a new __collection__ and add a __HTTP POST request__.<br/><br/>
    - Copy the __Invoke URL__ from API Gateway and paste it in Postman.<br/><br/>
    - In the Body tab, select raw JSON format and type:

In [ ]:
{
    "age": "40",
    "height": "190",
    "income": "30000"
}

- Click __Send__.<br /><br />
- Check for `status 200` and switch to __DynamoDB__ and check the items of the table __`compare-yourself-[CODE]`__. The values sent from the __Postman__ must be reflected.

### II. Create a custom policy in AWS IAM

1. Go to AWS `IAM > Access Management > Policies > Create Policy`<br /><br />
2. Select __DynamoDB__ service.<br /><br />
3. In __Read__ access level, check __GetItem__ and __Scan__.<br /><br />
4. Select __Specific__ resource and click __Add ARN__ (Amazon Resource Name) for __index__.<br /><br />
5. Click __List ARNs manually__ and paste the ARN for __`compare-yourself-[CODE]`__ table.<br /><br />
6. Click Next: Tags.<br /><br />
7. Click Next: Review.<br /><br />
8. Name the policy as __`dynamodb-scan-cy-role-[CODE]`__.<br /><br />
9. Click __Create policy__.<br /><br />
10. Attach the __`dynamodb-scan-cy-role-[CODE]`__ policy to the Lambda execution role for the __`cy-get-data-[CODE]`__ function.
    - Search for __`dynamodb-scan-cy-role-[CODE]`__.
    - Select it and click __Attach__ under the __Actions__ menu.
    - Search for the execution role for __`cy-get-data-[CODE]`__ function.
    - Select it and click __Attach policy__.

### III. Link the API GET method and its Lambda function to the DynamoDB table.

1. In Lambda, modify the __`cy-get-data-[CODE]`__ function as bellow:

In [ ]:
const AWS = require('aws-sdk');
const dynamodb = new AWS.DynamoDB({region:'sa-east-1',apiVersion:'2012-08-10'});

exports.handler = (event, context, callback) => {
    const type = event.type;
    
    if (type === 'all') {
        const params = {
            TableName:'compare-yourself-1mtr15'
        }
        dynamodb.scan(params, function(err, data) {
            if (!err) {
                console.log(data);
                callback(null, '[OK] ' + data)
            } else {
                console.log(err);
                callback(err);
            }
        })
    } else if (type === 'single') {
        callback(null, 'single');
    } else {
        callback(null, 'other');
    }
}

5. __Deploy__ the function.<br /><br />
6. Create a __test event__ with the following JSON format:

In [ ]:
{
    "type": "all"
}

7. Click __Test__ and review the results.

<div class="alert alert-block alert-info">
In the current state of our code, the items returned are <b>serialized</b> so we cannot see the actual values. To change this behaviour we need to <b>deserialize</b> the data returned from DynamoDB.
</div>

8. Modify the __`cy-get-data-[CODE]`__ function as bellow to deserialize the data retrieved from DynamoDB:

In [ ]:
const AWS = require('aws-sdk');
const dynamodb = new AWS.DynamoDB({region:'sa-east-1',apiVersion:'2012-08-10'});

exports.handler = (event, context, callback) => {
    const type = event.type;
    
    if (type === 'all') {
        const params = {
            TableName:'compare-yourself-1mtr15'
        }
        dynamodb.scan(params, function(err, data) {
            if (!err) {
                console.log(data);
                
                const items = data.Items.map {
                    (dataField) => {
                        return {Age: + dataField.Age.N,
                               Height: + dataField.Height.N,
                               Income: + dataField.Income.N}
                    }
                }
                
                callback(null, items)
            } else {
                console.log(err);
                callback(err);
            }
        })
    } else if (type === 'single') {
        callback(null, 'single');
    } else {
        callback(null, 'other');
    }
}

9. __Deploy__ the function.<br /><br />
10. Click __Test__ and review the results.<br /><br />
11. Modify the __`cy-get-data-[CODE]`__ to complete the `single` condition using a hard value.

In [ ]:
const AWS = require('aws-sdk');
const dynamodb = new AWS.DynamoDB({region:'sa-east-1',apiVersion:'2012-08-10'});

exports.handler = (event, context, callback) => {
    const type = event.type;
    
    if (type === 'all') {
        const params = {
            TableName:'compare-yourself-1mtr15'
        }
        dynamodb.scan(params, function(err, data) {
            if (!err) {
                console.log(data);
                
                const items = data.Items.map {
                    (dataField) => {
                        return {Age: + dataField.Age.N,
                               Height: + dataField.Height.N,
                               Income: + dataField.Income.N}
                    }
                }
                
                callback(null, items)
            } else {
                console.log(err);
                callback(err);
            }
        })
    } else if (type === 'single') {
        const params = {
            Key: {
                "Userid": {
                    S: "U..."  // Copy a Userid value from your table 
                }
            },
            TableName:'compare-yourself-1mtr15'
        }
        
        dynamodb.getItem(params, function(err, data) {
            if (!err) {
                console.log(data);
                // Return an array
                callback(null, [Age: + data.Item.Age.N,
                               Height: + data.Item.Height.N,
                               Income: + data.Item.Income.N])
            } else {
                console.log(err);
                callback(err);
            }
        }
    } else {
        callback(null, 'other');
    }
}

11. __Deploy__ the function.<br /><br />
12. Create a __test event__ with the following JSON format:

In [ ]:
{
    "type": "single"
}

12. Click __Test__ and review the results.<br /><br />
13. Perform persistence tests from __API Gateway__ and __Postman__:<br /><br />
    13.1. __Test__ from __API Gateway__:
    - Go to __GET__ method and click __Test__.<br /><br />
    - In the `{type}` path, type `all` and click __Test__.<br /><br />
    - Review the results.<br /><br />
    - Repeat the previous steps with the type `single`.<br /><br />
    13.2. __Test__ from __Postman__:
    - __Deploy__ API.<br/><br/>
    - Open Postman, create a new __HTTP GET request__.<br/><br/>
    - Copy the __Invoke URL__ from API Gateway and paste it in Postman.<br/><br/>
    - Add `/all` and `/single` for each test.

### IV. Create a custom authorization

1. Go to __`Lambda > Create function`__.<br /><br />
2. Create a function named __`cy-custom-auth-[CODE]`__ leaving the remaining fields with their default values.

In [ ]:
exports.handler = (event, context, callback) => {
    const token = event.authorizationToken;
    
    if (token === 'allow') {
        const policy = generatePolicy('allow', event.methodArn);
        const principalId = 'abcd1234';
        const context = {
            simpleAuth: true
        }
        const response = {
            principalId: principalId,
            policyDocument: policy,
            context: context
        }
        callback(null, response);
    } else if (token === 'deny') {
        const policy = generatePolicy('deny', event.methodArn);
        const principalId = 'abcd1234';
        const context = {
            simpleAuth: true
        }
        const response = {
            principalId: principalId,
            policyDocument: policy,
            context: context
        }
        callback(null, response);
    } else {
        callback('Unauthorized');
    }
}

function generatePolicy(effect, resource) {
    const policy = {};
    policy.Version = '2012-10-17';
    policy.Statement = [];
    
    const stmt = {};
    stmt.Action = 'execute-api:Invoke';
    stmt.Effect = effect;
    stmt.Resource = resource;
    
    policy.Statement.push(stmt);
    return policy;
}

3. __Deploy__ the function.<br /><br />
4. Configure a new __test event__ with the following JSON format:

In [ ]:
{
    "authorizationToken": "allow"
}

5. Review the results.<br /><br />
6. Go to __`API Gateway > Authorizers`__ and click __Create New Authorizer__.<br /><br />
7. Create a new authorizer with the following attributes:<br />
    - Name: __`simple-custom-auth-[CODE]`__
    - Type: __Lambda__
    - Lambda function: __`cy-custom-auth-[CODE]`__
    - Lambda Event Payload: __Token__
    - Token source: __`Authorization`__<br /><br />
8. Click __Test__.<br /><br />
9. Type `allow` as Authorization (header).<br /><br />
10. Review the results.<br /><br />
11. Go to __`API Gateway > Resources > POST > Method Request`__.<br /><br />
12. Select `simple-custom-auth-[CODE]`as Authorization and click Update.<br /><br />
13. __Deploy__ API.<br /><br />
14. Go to __Postman__, in the __POST__ request select the __Headers__ tab and add the key-value pair `Authorization:allow`<br /><br />
15. Review the results.<br /><br />
16. As another test, change the key-value pair to `Authorization:deny`<br /><br />
17. Review the results.

POST > Integration Request > Mapping Templates

In [ ]:
{
    
    "age": $input.json('$.age'),
    "height": $input.json('$.height'),
    "income": $input.json('$.income'),
    "userId": "$context.authorizer.principalId"
}

#### Lambda: cy-store-data-[code]

In [ ]:
const AWS = require('aws-sdk');
const dynamodb = new AWS.DynamoDB({region:'sa-east-1',apiVersion:'2012-08-10'});

exports.handler = (event, context, callback) => {
    const params = {
        Item: {
            "Userid": {
                "S": event.userId
            },
            "Age": {
                "N": "" + event.age
            },
            "Height": {
                "N": "" + event.height
            },
            "Income": { 
                "N": "" + event.income
            }
        },
        TableName: "compare-yourself-1mtr15"
    }

    dynamoDB.putItem(params, function(err, data) {
        if (!err) {
            console.log(data);
            callback(null, data);
        } else {
            console.log(err);
            callback(err);
        }
    })
}

### V. Build a front end web app hosted on GitHub

### VI. Hosting a website in S3

1. Go to __Amazon S3__.<br /><br />
2. Create a bucket with the following attributes:<br />
    - Bucket name: __`[CODE].cy`__
    - AWS Region: __`US East (Ohio) us-east-2`__
    - In the section Block Public Access settings for this bucket, __Uncheck__ _Block all public access_
    - __Check__ _I acknowledge that the current settings might result in this bucket and the objects within becoming public._<br /><br />
3. Open the bucket __`[CODE].cy`__<br /><br />
4. Upload all the files located in the `dist` directory of the Angular project.<br /><br />
5. Switch to the __Permissions__ tab.<br /><br />
6. In the __Bucket policy__ section, click __Edit__<br /><br />
7. Type the following __policy__:
    

In [ ]:
{
    "Version": "2012-10-17"
    "Statement": [
        {
            "Sid": "PublicRead",
            "Principal": "*",
            "Effect": "Allow",
            "Action": ["s3:GetObject","s3:GetObjectVersion"],
            "Resource": ["arn:aws:s3:.../*"]  // Type bucket ARN (add /*)
        }
    ]
}

8. Click __Save changes__.<br /><br />
9. Switch to the __Properties__ tab.<br /><br />
10. In the __Static website hosting__ section, click __Edit__.<br /><br />
11. __Enable__ Static website hosting.<br /><br />
12. Type __`index.html`__ as Index document and Error document.<br /><br />
13. Click __Save changes__.<br /><br />
14. Now the __URL__ of the static website is displayed at the bottom of the page.